In [ ]:
from transformers import AutoTokenizer, AutoModel
import numpy as np

# Load tokenizer and model
tokenizer = AutoTokenizer.from_pretrained("ProsusAI/finbert")
model = AutoModel.from_pretrained("ProsusAI/finbert")

# Example document with more than 512 tokens
document = "Long document with more than 512 tokens..."

# Split the document into chunks of 512 tokens
max_length = 512
document_chunks = [document[i:i+max_length] for i in range(0, len(document), max_length)]

# Create embeddings for each chunk
document_embeddings = []
for chunk in document_chunks:
    encoded_input = tokenizer(chunk, padding=True, truncation=True, max_length=max_length, return_tensors='pt')
    with torch.no_grad():
        model_output = model(**encoded_input)
        embeddings = model_output.last_hidden_state[:, 0, :].numpy()
        document_embeddings.append(embeddings)

# Aggregate embeddings to create an overall embedding for the document
document_embedding = np.mean(document_embeddings, axis=0)


In [1]:
!pip install transformers
!pip install torch
!pip install annoy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 37.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 18.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 3.4 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 647.4/647.4 kB 13.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for annoy: filename=annoy-1.17.2-cp310-cp310-linux_x86_64.whl size=582722 sha256=b9d9c1c73f290b594407d7784a56dd5ce63c0567400716262ae7b6038b749b31
  Stored in directory: /root/.cache/pip/wheels/7a/d9/59/473fa56df8e39430eeda369500b4e7127f5b243ba24c3c4297
Successfully built annoy


In [6]:
import torch
from transformers import AutoTokenizer, AutoModel
from annoy import AnnoyIndex

# Load the model and tokenizer
model = AutoModel.from_pretrained("ProsusAI/finbert")
tokenizer = AutoTokenizer.from_pretrained("ProsusAI/finbert")

# Define the maximum length for document tokens
max_length = 512

# Define the number of trees for the Annoy index
num_trees = 10

# Define the dimensionality of the embeddings
embedding_size = 768

# Define the sliding window size and overlap
window_size = 256
overlap = 128

# Define the list to store the document embeddings
document_embeddings = []

# Define a function to create document embeddings with sliding windows
def create_document_embeddings(document):
    # Tokenize the document
    tokens = tokenizer.tokenize(document)
    # Create sliding windows of the document
    windows = [tokens[i:i+window_size] for i in range(0, len(tokens), window_size-overlap)]
    # Create document embeddings for each window
    embeddings = []
    for window in windows:
        # Encode the window with the tokenizer
        inputs = tokenizer(" ".join(window), max_length=max_length, padding='max_length', truncation=True, return_tensors='pt')
        # Pass the inputs through the model to get the embeddings
        with torch.no_grad():
            output = model(**inputs)
            embeddings.append(output.pooler_output.cpu().numpy()[0])
    # Concatenate the embeddings to get the document embedding
    document_embedding = np.concatenate(embeddings)
    return document_embedding

# Create the document embeddings
documents = ["your client can use the following funding tooltip","schwab offers three main resources"] # list of documents
for document in documents:
    document_embedding = create_document_embeddings(document)
    document_embeddings.append(document_embedding)

# Build the Annoy index
annoy_index = AnnoyIndex(embedding_size, 'angular')
for i in range(len(document_embeddings)):
    annoy_index.add_item(i, document_embeddings[i])
annoy_index.build(num_trees)

# Define a function to perform semantic search
 

Some weights of the model checkpoint at ProsusAI/finbert were not used when initializing BertModel: ['classifier.bias', 'classifier.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


True

In [7]:
import numpy as np
def semantic_search(query, k):
    # Encode the query with the tokenizer
    inputs = tokenizer(query, max_length=max_length, padding='max_length', truncation=True, return_tensors='pt')
    # Pass the inputs through the model to get the query embedding
    with torch.no_grad():
        query_embedding = model(**inputs).pooler_output.cpu().numpy()[0]
    # Use the Annoy index to retrieve the most similar documents
    indices, distances = annoy_index.get_nns_by_vector(query_embedding, k, include_distances=True)
    # Return the indices and distances
    return indices, distances


In [10]:
semantic_search("cient fund account after opening",2)

([0, 1], [0.31801414489746094, 0.37996208667755127])